In [49]:
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load
from tqdm.notebook import tqdm



In [ ]:
#pipelines 
alors = ["kraken", "tesseract"]
types = ['train','test','val']
alors_counter = 0

In [11]:

store = {}
for al in alors:
    for ty in types:
        store["{}_{}".format(al,ty)] = load_files("output/{}/{}".format(al,ty))



In [20]:
parameters_Multinomial = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
    ])


#--------------------------------------------

text_sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()), 
    ])

parameters_sgd = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    "clf__loss" : ["hinge", "log", "squared_hinge", "modified_huber", "perceptron"],
    "clf__alpha" : [0.0001, 0.001, 0.01, 0.1],
    "clf__penalty" : ["l2", "l1", "elasticnet", "none"],
    }

#-------------------------------------------

text_rf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
    ])

parameters_rf = { 
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__n_estimators': [200, 500],
    'clf__max_features': ['auto', 'sqrt', 'log2'],
    'clf__max_depth' : [4,5,6,7,8,10,20,50,100],
    'clf__criterion' :['gini', 'entropy']
    }

In [51]:
algor_list = [(parameters_Multinomial, text_clf), (parameters_rf ,text_rf)]
#, ( parameters_sgd ,text_sgd)
algor_name  = ["MultinomialNB", "RandomForest"]
#, "SGDClassifier"
best_parameters_store ={}

In [36]:
hold = store["{}_train".format("kraken")]

In [ ]:
counter = 0

for search in  tqdm(algor_list):
    
    for al in tqdm(alors):
    
        gs = GridSearchCV(search[1], search[0], cv = 5, n_jobs = -1)

        model = gs.fit(store["{}_train".format(al)].data, store["{}_train".format(al)].target )
        
        best_parameters_store['{}_{}_{}'.format(al, algor_name[counter]), alors_couter] = model
        
        dump(model, "models/{}_{}.joblib".format(al, algor_name[counter]))
        
        print("For algorithm {} model: {} \n Accurancy {} \n best model {}".format(al , algor_name[counter] , model.best_score_ , model.best_params_ ))
        
    counter += 1
    alors_counter += 1
    
        

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

For algorithm kraken model: MultinomialNB 
 Accurancy 0.4431 
 best model {'clf__alpha': 0.01, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
For algorithm tesseract model: MultinomialNB 
 Accurancy 0.649 
 best model {'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}


  0%|          | 0/2 [00:00<?, ?it/s]

For algorithm kraken model: RandomForest 
 Accurancy 0.36019999999999996 
 best model {'clf__criterion': 'entropy', 'clf__max_depth': 8, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
